In [7]:
from pymongo import MongoClient
from config import *

client = MongoClient()
client = MongoClient(connectionString)
database = client['masters']
database.rates

Collection(Database(MongoClient(host=['masters-shard-00-00.cpq5u.mongodb.net:27017', 'masters-shard-00-01.cpq5u.mongodb.net:27017', 'masters-shard-00-02.cpq5u.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-wrhudr-shard-0', tls=True), 'masters'), 'rates')

In [108]:
def get_rates_by_user(user):
    cursor = database.rates.aggregate([
        {
            "$unwind": "$reviews"
        },
        {
            '$match' : {'reviews.user._id' : user}
        },
        {
            '$project': {
                'bookId': '$bookId',
                'rate': '$reviews.rate'
                }
        }
    ])
    return list(cursor)    

def get_books_by_ids(bookIds):
    cursor = database.books.aggregate([
        {
            '$match' : {'_id' : { '$in' : bookIds }}
        },
        {
            '$project': {
                'title': '$title',
                'description': '$description',
                'genres': '$genres.name',
                'author': '$author.name'
                }
        }
    ])
    return list(cursor)

def get_books_not_in_ids(bookIds):
    cursor = database.books.aggregate([
        {
            '$match' : {'_id' : { '$nin' : bookIds }}
        },
        {
            '$project': {
                'title': '$title',
                'description': '$description',
                'genres': '$genres.name',
                'author': '$author.name'
                }
        }
    ])
    return list(cursor)

def get_books():
    cursor = database.books.aggregate([
        {
            '$project': {
                'title': '$title',
                'description': '$description',
                'genres': '$genres.name',
                'author': '$author.name'
                }
        }
    ])
    return list(cursor)

In [109]:
from bson.objectid import ObjectId
import pandas as pd

user_rates = get_rates_by_user(ObjectId("620bb9fef23b1bc78052c5e6"))
user_rates

rates_df = pd.DataFrame(user_rates)
rates_df.drop("_id", axis=1, inplace=True)
bookdIds = list(rates_df.bookId)

books = get_books()#[:5]
books_df = pd.DataFrame(books)

main_df = pd.merge(books_df, rates_df, how="left", left_on="_id", right_on="bookId")
main_df.drop("bookId", axis=1, inplace=True)
main_df.genres = [' '.join(map(str, l)) for l in main_df.genres]

columns_to_join = ["title", "description", "genres"]
main_df['full_desc'] = main_df[columns_to_join].T.agg(' '.join)
main_df.drop(columns_to_join, axis=1, inplace=True)
main_df

,_id,author,rate,full_desc
0,630602ac073e94de49556747,Hajime Isayama,NaN,Attack on Titan: Volume 13 NO SAFE PLACE LEFT ...
1,630602ac073e94de4955674d,J.M. Barlog,NaN,God of War: The Official Novelization The nove...
2,630602ac073e94de4955674b,Peter V. Brett,NaN,"The Painted Man (The Demon Cycle, Book 1) The ..."
3,630602ac073e94de49556751,Andrzej Sapkowski,4.0,Sword of Destiny: Witcher 2: Tales of the Witc...
4,630602ac073e94de49556758,Story by FUSE,NaN,That Time I got Reincarnated as a Slime 11 Mik...
...,...,...,...,...
241,630602ac073e94de49556820,Alvin Hall,NaN,The Secrets of Saving and Investing with Alvin...
242,630602ac073e94de495567f3,Voltaire,NaN,Candide Voltaire’s satire of eighteenth-centur...
243,630602ac073e94de4955680a,Blake Pierce,NaN,Once Missed (A Riley Paige Mystery—Book 16) “A...
244,630602ac073e94de49556818,André Aciman,NaN,Find Me: A TOP TEN SUNDAY TIMES BESTSELLER In ...


In [110]:
import string, nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

def text_preprocess(sample):
    # lower-casing
    lower_text = sample.lower()

    entry_no_punct = lower_text.translate(str.maketrans('', '', string.punctuation))

    # word-level tokenization
    words = word_tokenize(entry_no_punct)

    # stopwords removal
    stops = stopwords.words('english')
    clean = [word for word in words if word not in stops]

    ps = PorterStemmer()
    # using Porter stemmer to stem our sentence
    stemmed_list = [ps.stem(w) for w in clean]

    return ' '.join(stemmed_list)

In [111]:
main_df["full_desc"] = main_df['full_desc'].apply(text_preprocess)
main_df

,_id,author,rate,full_desc
0,630602ac073e94de49556747,Hajime Isayama,NaN,attack titan volum 13 safe place left great co...
1,630602ac073e94de4955674d,J.M. Barlog,NaN,god war offici novel novel highli anticip god ...
2,630602ac073e94de4955674b,Peter V. Brett,NaN,paint man demon cycl book 1 stun debut fantasi...
3,630602ac073e94de49556751,Andrzej Sapkowski,4.0,sword destini witcher 2 tale witcher geralt wi...
4,630602ac073e94de49556758,Story by FUSE,NaN,time got reincarn slime 11 mikami ’ middl age ...
...,...,...,...,...
241,630602ac073e94de49556820,Alvin Hall,NaN,secret save invest alvin hall simpl strategi m...
242,630602ac073e94de495567f3,Voltaire,NaN,candid voltair ’ satir eighteenthcenturi europ...
243,630602ac073e94de4955680a,Blake Pierce,NaN,miss riley paig mystery—book 16 “ masterpiec t...
244,630602ac073e94de49556818,André Aciman,NaN,find top ten sunday time bestsel find aciman s...


In [112]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
  
main_df['author']= label_encoder.fit_transform(main_df['author'])

# mask_var = main_df['rate'].isnull()  
mask = main_df.index.isin(main_df.index.values) & main_df.rate.isnull()
main_df.sort_values('rate', inplace=True)
display(main_df)

test_amount = len(main_df[mask])
print(test_amount)

,_id,author,rate,full_desc
15,630602ac073e94de49556761,162,1.0,introduc critic theori graphic guid last decad...
98,630602ac073e94de4955676b,134,1.0,daylight war book three demon cycl new york ti...
232,630602ac073e94de49556836,173,1.0,injustic god among us year one complet collect...
5,630602ac073e94de49556774,121,2.0,youll never see daylight michaella regular gir...
20,630602ac073e94de49556791,51,3.0,perman record new york time bestsel edward sno...
...,...,...,...,...
241,630602ac073e94de49556820,5,NaN,secret save invest alvin hall simpl strategi m...
242,630602ac073e94de495567f3,176,NaN,candid voltair ’ satir eighteenthcenturi europ...
243,630602ac073e94de4955680a,14,NaN,miss riley paig mystery—book 16 “ masterpiec t...
244,630602ac073e94de49556818,7,NaN,find top ten sunday time bestsel find aciman s...


232


C:\Users\User\AppData\Local\Temp\ipykernel_1676\1539904245.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_amount = len(main_df[mask])


In [113]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

def transform(text):
    cv_transf = CountVectorizer()
    tfidf_transf = TfidfTransformer()
    cv_transf.fit(text)
    cv_metrix = cv_transf.transform(text)
    tfidf_transf.fit(cv_metrix)
    result = tfidf_transf.transform(cv_metrix)
    return result

In [114]:
from scipy.sparse import hstack
import numpy as np

data = transform(main_df["full_desc"])
data = hstack((data,np.array(main_df['author'])[:,None]))

data

<246x6316 sparse matrix of type '<class 'numpy.float64'>'
	with 22544 stored elements in COOrdinate format>

In [115]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

X_train, X_test, y_train, y_test = train_test_split(data, 
                                        main_df['rate'], test_size=test_amount,
                                        shuffle=False)

print(X_train.toarray())

trained_model = model.fit(X_train.toarray(), y_train)
pred = trained_model.predict(X_test.toarray())

[[  0.   0.   0. ...   0.   0. 162.]
 [  0.   0.   0. ...   0.   0. 134.]
 [  0.   0.   0. ...   0.   0. 173.]
 ...
 [  0.   0.   0. ...   0.   0. 181.]
 [  0.   0.   0. ...   0.   0. 146.]
 [  0.   0.   0. ...   0.   0.   1.]]


In [116]:
result_df = pd.DataFrame()
result_df["id"] = main_df[mask]["_id"]
result_df["expected_rate"] = pred

C:\Users\User\AppData\Local\Temp\ipykernel_1676\1512593405.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result_df["id"] = main_df[mask]["_id"]


In [117]:
result_df.sort_values('expected_rate', ascending=False, inplace=True)
result_df

,id,expected_rate
86,630602ac073e94de495567a2,5.0
115,630602ac073e94de49556762,5.0
107,630602ac073e94de495567bb,5.0
200,630602ac073e94de49556815,5.0
203,630602ac073e94de495567ec,5.0
...,...,...
104,630602ac073e94de4955676d,1.0
103,630602ac073e94de495567d0,1.0
101,630602ac073e94de495567c1,1.0
100,630602ac073e94de4955679c,1.0


In [103]:
from sklearn.metrics.pairwise import cosine_similarity

result = cosine_similarity(data, [data.toarray()[0]])
len(result)

246

In [104]:
data.todense()[0]

matrix([[  0.,   0.,   0., ...,   0.,   0., 162.]])

In [ ]:
from sklearn import linear_model
from sklearn import tree
from sklearn.linear_model import LogisticRegression

# model = linear_model.BayesianRidge()
model = linear_model.LogisticRegression()
# model = linear_model.TweedieRegressor()
# model = tree.DecisionTreeRegressor()

# train
trained_model = model.fit(X_train.toarray(), y_train)

# get all label predictions
pred = trained_model.predict(X_test)

pred

In [ ]:
X_train.toarray()

In [124]:
text_examples = ["Fiction, Adventure, Action",
"General, Fantasy, Fiction, Mystery",
"Fantasy, Adventure, Fiction",
"Thrillers, Detective, Mystery, Adventure",
"Alien Contact, Superheroes, Adventure"]

result_text = [text_preprocess(text) for text in text_examples]
result_text

def transform(text):
    cv_transf = CountVectorizer()
    cv_transf.fit(text)
    cv_metrix = cv_transf.transform(text)
    return cv_metrix

a = transform(result_text)
a.toarray()

array([[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0]], dtype=int64)

In [ ]:
mask_var = main_df['rate'].isnull()  

# test_df = main_df[mask_var]  
# test_df

# train_df = main_df[~mask_var]  
# train_df

main_df.sort_values('rate', inplace=True)
main_df
test_amount = main_df[mask_var]  

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

model = MultinomialNB()

def transform(data):
    cv_transf = CountVectorizer()
    tfidf_transf = TfidfTransformer()
    cv_transf.fit(data)
    cv_metrix = cv_transf.transform(data)
    tfidf_transf.fit(cv_metrix)
    result = tfidf_transf.transform(cv_metrix)
    return result

In [ ]:
from scipy.sparse import hstack
import numpy as np

train_data = transform(train_df["full_desc"])
train_data = hstack((train_data,np.array(train_df['author'])[:,None]))
train_data.todense()

test_data = transform(test_df["full_desc"])
test_data = hstack((test_data,np.array(test_df['author'])[:,None]))
test_data.todense()

In [ ]:
# split the entire dataset in train and test cuts: 50% and 50% respectively    
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, 
                                        df1['sentiment'], test_size=test_amount)

# train
trained_model = model.fit(train_data, train_df["rate"])

# get all label predictions
pred = trained_model.predict(test_data)

pred

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
X, y = load_iris(return_X_y=True)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
# gnb = GaussianNB()
# y_pred = gnb.fit(X_train, y_train).predict(X_test)
X

In [10]:
import pandas as pd
df = pd.DataFrame(["1", "2", "3", "4", "5"])
a = pd.get_dummies(df).values
a

array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1]], dtype=uint8)